In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from cl.eval import EvalReader as EPPS
import seaborn as sns


In [ ]:
def common_parse(name, add):
    if name.find("pertask")>=0:
        _, tid, fold = name.split("_")
        name = "{} on fold {}".format(tid, fold)
    return "Result of Task {} {}:".format(name, add)
def listdct_to_dct(lst, kname, vname):
    ret = {}
    for d in lst:
        ret[tuple(d[kname])] = d[vname]
    return ret
def parse_pairwise_ic(name, measure):
    tname = common_parse(name, "Pairwise Inconsistency")
    print(tname)
    pairwise = listdct_to_dct(measure["inconsist_pairwise"], kname="compare", vname="consistency")
    idx = measure["index"]
    l = len(idx)
    _idx_name = ["Model {}".format(i) for i in idx]
    _df = pd.DataFrame(index = _idx_name[:-1], columns = _idx_name[1:]).fillna("")
    for i in range(l):
        for j in range(i+1, l):
            _df.loc[_idx_name[i], _idx_name[j]] = pairwise[(idx[i],idx[j])]
    #_df.style.set_caption(tname)
    print(_df)
    return _df

def parse_acc(name, measure):
    print(common_parse(name, "Accuracy Per Step"))
    for i in range(len(measure["index"])):
        print(measure["acc_list"][i], end="\t")

def parse_total(name, measure):
    print(common_parse(name, "Inconsistency Together"))
    print(measure["inconsist_tot"])
    return measure["inconsist_tot"]

In [ ]:
def read_result(evaluator, bname):
    global rec
    e = evaluator
    print("="*50)
    rec[bname] = []
    for name in e.names[:-3]:
        parse_pairwise_ic(name, e._measure_pairwise(name))
        print()
        parse_acc(name,e._measure_acc(name))
        print()
        tot = parse_total(name, e._measure([name]))
        true_name = "_".join(name.split("_")[-2:])
        rec[bname].append((true_name,tot))
        print(tot)
def draw_pic(picname):
    names = ["vanilla","lwf", "ewc"]
    plt.figure(figsize=(10,10))
    tasks = []
    vals = []
    comps = []
    print(rec)
    for name in names:
        for v in rec[name]:
            task, val = v
            comps.append(name)
            tasks.append(task)
            vals.append(val)
    df = pd.DataFrame({"Task":tasks, "Method":comps, "Inconsistency":vals})
    pd.melt(df, id_vars=['Task'], value_vars=['Inconsistency'])
    sns.barplot(x = 'Task', y = 'Inconsistency', data = df, hue="Method")
    plt.ylabel('Inconsistency')
    plt.title('Inconsistency over different baselins');
    plt.show()
    plt.savefig("Inconsistency_{}.png".format(picname))

In [ ]:
epps = EPPS()
for i in range(5):
    rec = {}
    #epps.load("results/cifar10_5_domain_inc")
    epps.load("results/cifar10_ResNet18_#vanilla_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed1")
    read_result(epps, "vanilla")
    epps.load("results/cifar10_ResNet18_#lwf1.00e+00_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed1")
    read_result(epps, "lwf")
    epps.load("results/cifar10_ResNet18_#ewc1.00e+00_DScifar10_CIMmask_CTdomain_inc_DAsequential_CvgD9.00e-01_CvgT1.000000e-03_DomS5/Seed1")
    read_result(epps,"ewc")
    draw_pic("Domain_Inc_seed_{}".format(i))

In [ ]:
print(epps.names)
print(epps.hist_version["pertask_0_test"][:10,:10])